## Лабораторна робота 7 з ІСППР
### Виконали студенти групи КІ-31мп Шабо О.А. та Сотник Д.C.
### Варіант 8 арифметичної нормалізації, HCR, мультиплікативний

![6.png](Images\\6.png)
![7.png](Images\\7.png)

# Imports

In [1]:
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from copy import deepcopy
import numpy as np

# Model training using grid search and metrics calculation

In [2]:
# Load dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

RAND_CONST = 10

# Perform stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RAND_CONST)

# Define models
models = {
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=RAND_CONST),
    'svm': SVC(random_state=RAND_CONST),
    'random_forest': RandomForestClassifier(random_state=RAND_CONST),
    'knn': KNeighborsClassifier(),
    'decision_tree': DecisionTreeClassifier(random_state=RAND_CONST)
}

# Define hyperparameters for each model
params = {
    'logistic_regression': {'classifier__C': [0.01, 0.1, 1, 10]},
    'svm': {'classifier__C': [0.01, 0.1, 1, 10], 'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'classifier__gamma': ['scale', 'auto']},
    'random_forest': {'classifier__n_estimators': [50, 100, 200, 350], 'classifier__max_depth': [None, 5, 10, 15, 20], 'classifier__min_samples_split': [2, 5, 10]},
    'knn': {'classifier__n_neighbors': [3, 5, 7, 9, 11], 'classifier__weights': ['uniform', 'distance'], 'classifier__p': [1, 2]},
    'decision_tree': {'classifier__max_depth': [None, 5, 10, 15, 20], 'classifier__min_samples_split': [2, 5, 10]}
}

# Perform GridSearchCV and evaluate models
metrics = {'f1': f1_score, 'roc_auc': roc_auc_score}
results = {metric: {} for metric in metrics}

for model in tqdm(models, total=len(models)):
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=15)),
        ('classifier', models[model])
    ])
    grid = GridSearchCV(pipe, params[model], cv=5)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    for metric in metrics:
        results[metric][model] = metrics[metric](y_test, y_pred)  # Evaluate on the testing set

# Normalize metric values to get weight vectors
results_scaled = deepcopy(results)
for metric in results:
    values = np.array(list(results[metric].values()))
    values_scaled = values / np.sum(values)
    results_scaled[metric] = {model: weight for model, weight in zip(models, values_scaled)}

  0%|          | 0/5 [00:00<?, ?it/s]

Metric results f1_score and roc_auc_score for the five models

In [3]:
results

{'f1': {'logistic_regression': 0.965034965034965,
  'svm': 0.9659863945578231,
  'random_forest': 0.951048951048951,
  'knn': 0.9444444444444444,
  'decision_tree': 0.9115646258503401},
 'roc_auc': {'logistic_regression': 0.9553571428571429,
  'svm': 0.945436507936508,
  'random_forest': 0.9365079365079365,
  'knn': 0.9246031746031746,
  'decision_tree': 0.8700396825396826}}

## Local weights

In [4]:
results_scaled

{'f1': {'logistic_regression': 0.2036764029150177,
  'svm': 0.20387720780796362,
  'random_forest': 0.20072457098871313,
  'knn': 0.19933065035684705,
  'decision_tree': 0.19239116793145863},
 'roc_auc': {'logistic_regression': 0.20625401584921824,
  'svm': 0.20411222959948597,
  'random_forest': 0.20218462197472692,
  'knn': 0.19961447847504818,
  'decision_tree': 0.18783465410152067}}

![1.png](Images\\1.png)
![2.png](Images\\2.png)
![3.png](Images\\3.png)

In [5]:
def HM(weights):
    return len(weights) / np.sum(weights)
def HCI(weights):
    return (HM(weights) - len(weights)) * (len(weights) + 1) / (len(weights) * (len(weights) - 1))
def HCR(weights):
    return HCI(weights) / 1.11 # we divide by 1.11 since n=5
def test_coherence(comparison_matrix):
    column_sums = np.sum(comparison_matrix, axis=0)
    weights = 1 / column_sums
    normalized_weights = weights / np.sum(weights) # Normalize the weights so they sum to 1
    indicator = HCR(weights)
    print("HCRN is coherent") if indicator <= 0.1 else print("HCRN isn't coherent") # the threshold is 0.1 since n=5
    return normalized_weights, indicator

In [6]:
# Define a matrix of comparisons for the "ability to tweak model to your needs" of each model
comparison_matrix = np.array([
    [1  , 1/2, 1/4, 2  , 1/3],
    [2  , 1  , 1/3, 3  , 4  ],
    [4  , 3  , 1  , 5  , 3  ],
    [1/2, 1/3, 1/5, 1  , 1/4],
    [3  , 1/4, 1/3, 4  , 1  ]
])
weights_for_subjective_criterion, HCR_value = test_coherence(comparison_matrix)

HCRN is coherent


In [7]:
weights_for_subjective_criterion

array([0.10050752, 0.20760569, 0.49858059, 0.07035526, 0.12295094])

In [8]:
HCR_value

0.07476880943311598

As we see the HCR_value = 0.074 < 0.1, so HCRN is coherent
## Global weights
![4.png](Images\\4.png)
![5.png](Images\\5.png)

In [9]:
criterion_weights = np.array([0.45, 0.35, 0.2]) # weights for the three criteria
# first: f1
# second: roc_auc
# third: subjective criterion
def find_global_weights(results_scaled, weights_for_subjective_criterion, criterion_weights):
    local_weights_matrix = np.zeros((len(criterion_weights), len(weights_for_subjective_criterion)))
    for ind, metric in enumerate(results_scaled): # first two rows correspond to numerical metrics
        metric_values = np.array(list(results_scaled[metric].values()))
        local_weights_matrix[ind, :] = metric_values
    local_weights_matrix[-1, :] = weights_for_subjective_criterion # third row corresponds to the subjective criterion
    global_weight = np.prod(np.power(local_weights_matrix.T, criterion_weights), axis=1)
    return local_weights_matrix.T, global_weight

In [11]:
local_weights_matrix_T, global_weights = find_global_weights(results_scaled, weights_for_subjective_criterion, criterion_weights)
for ind, model in enumerate(models):
    print(f"{model} global weight: {global_weights[ind]}")

logistic_regression global weight: 0.17762491169157274
svm global weight: 0.20470003366483328
random_forest global weight: 0.24139549345627243
knn global weight: 0.16193281557599146
decision_tree global weight: 0.17444205045820554


In [18]:
local_weights_matrix_T

array([[0.2036764 , 0.20625402, 0.10050752],
       [0.20387721, 0.20411223, 0.20760569],
       [0.20072457, 0.20218462, 0.49858059],
       [0.19933065, 0.19961448, 0.07035526],
       [0.19239117, 0.18783465, 0.12295094]])

### Models comparison

In [21]:
num_models = len(models)
model_counts = {model: 0 for model in models.keys()}

for i in range(num_models):
    for j in range(i+1, num_models):
        if np.prod(np.power(local_weights_matrix_T[i, :] / local_weights_matrix_T[j, :], criterion_weights)) >= 1:
            model_counts[list(models.keys())[i]] += 1
        else:
            model_counts[list(models.keys())[j]] += 1

# Sort models based on count
sorted_models = sorted(model_counts.items(), key=lambda x: x[1])

# Format output
output = " < ".join([model for model, count in sorted_models])
print(output)

knn < decision_tree < logistic_regression < svm < random_forest
